N.B. Ho osservato che nei file XML salvati nel mio computer a volte scomparivano degli spazi (es. 'dellaRenault' in luogo di 'della Renault' nel resoconto numero 9 della legislatura numero 18 o 'diroutine' in luogo di 'di routine' nel resoconto 18 della stessa legislatura), mi pare sempre prima di una parola che era contenuta in un tag \\<emphasis\>\. Ho debuggato passo passo e la scomparsa dello spazio non si verificava prima del metodo .write(), il che significa che è dovuta al metodo stesso o ad altro che non so come risolvere. Come soluzione di comodo, ho aggiunto uno spazio in testa al testo di ogni tag \\<emphasis\>\, il che sembra funzionare.

In [1]:
from urllib.request import urlopen
from lxml import etree
import re
import copy

In [6]:
regex_sigla = re.compile('\([^a-z]+?\) *\.?')
regex_virgola_punto = re.compile('\s*[,.]?\s*')
regex_punto = re.compile('\s*\.')

def pulitura_intervento(Element_intervento, attributi):
    testo_senza_tag = re.sub(" +"," ",re.sub('[\t\n]',"",re.sub('(?<!\t| )\n(?! )'," ",etree.tostring(Element_intervento, encoding='unicode', method='text'))))
    indentazione = re.search('[\t\n]+', Element_intervento.tail)[0]
    Element_intervento.clear()
    # per le quattro righe qua sotto si potrebbero usare invece le re
    Element_intervento.text = testo_senza_tag.strip()
    if Element_intervento.text[0] == '.' or Element_intervento.text[0] == ',':
        Element_intervento.text = Element_intervento.text[1:]
    Element_intervento.text = Element_intervento.text.strip()
    Element_intervento.tail = indentazione
    for key,value in attributi.items():
        Element_intervento.attrib[key] = value
    return

def gestione_intervento(Element_intervento, Element_padre):
    for Element in Element_intervento.iter('nominativo'):
        nodo_nominativo = Element
        break
    for Element_emphasis in Element_intervento.iter('emphasis'):
        Element_emphasis.text =" "+ Element_emphasis.text
    attributi = copy.deepcopy(Element_intervento.attrib)
    attributi['idNominativo'] = nodo_nominativo.attrib['id']
    attributi['cognomeNome'] = nodo_nominativo.attrib['cognomeNome']
    #intervento parlamentare
    stringa_controllo = nodo_nominativo.tail.strip()
    for token in stringa_controllo.split()[0:3]:
        if regex_sigla.match(token) is not None:
            sigla = regex_sigla.match(token)[0].strip(".")
            attributi['tipo'] = 'parlamentare'
            attributi['partito'] = sigla
            nodo_nominativo.tail = regex_sigla.sub("", nodo_nominativo.tail).strip()
            nodo_nominativo.text = None
            pulitura_intervento(Element_intervento, attributi)
            return
    # intervento parlamentare (troncato)
    if re.match(r' *?…', re.sub(r'[\t\n]',"", nodo_nominativo.tail)) is not None:
        lista_interventi_precedenti = list(list(Element_padre.iterancestors('seduta'))[0].iter('intervento'))
        lista_interventi_precedenti = lista_interventi_precedenti[lista_interventi_precedenti.index(Element_intervento)-4:lista_interventi_precedenti.index(Element_intervento)]
        lista_interventi_precedenti.reverse()
        for intervento_precedente in lista_interventi_precedenti:
            if intervento_precedente.attrib['tipo'] == 'parlamentare' and intervento_precedente.attrib['idNominativo'] == nodo_nominativo.attrib['id']:
                attributi['tipo'] = 'parlamentare'
                attributi['partito'] = intervento_precedente.attrib['partito'].strip('?') + '?'
                nodo_nominativo.text = None
                pulitura_intervento(Element_intervento, attributi)
                return
    attributi['tipo'] = 'altro'
    if sorted(nodo_nominativo.attrib['cognomeNome'].upper().split()) != sorted(nodo_nominativo.text.split()):
        attributi['nominativo'] = nodo_nominativo.text
    nodo_nominativo.tail = regex_punto.sub("",nodo_nominativo.tail, count=1)
    nodo_nominativo.text = None
    pulitura_intervento(Element_intervento, attributi)
    return

In [7]:
num_legislatura = str(18)
for num_seduta in range(1, 30):
    indirizzo_web = "http://documenti.camera.it/apps/resoconto/getXmlStenografico.aspx?idNumero=00{}&idLegislatura={}".format(num_seduta, num_legislatura)
    xml = urlopen(indirizzo_web).read()
    seduta = etree.fromstring(xml)
    # rimuove tag per la data estesa (inutile perché la data è già negli attributi di <seduta>)
    seduta[0].remove(seduta[0].find('dataEstesa'))
    tag_da_tenere = ['intervento', 'dibattito']
    for Element in seduta[1]:
        if Element.tag not in tag_da_tenere:
            seduta[1].remove(Element)
    tag_da_tenere = ['titolo','fase','intervento']
    file_da_salvare = False
    for Element in seduta[1]:
        if Element.tag == 'dibattito':
            da_salvare = False
            for Element2 in Element:
                if Element2.tag == 'intervento':
                    da_salvare = True
                    file_da_salvare = True
                    gestione_intervento(Element2, Element)    
                elif Element2.tag == 'fase':
                    fase_da_salvare = False
                    for Element3 in Element2:
                        if Element3.tag == 'intervento':
                            fase_da_salvare = True
                            da_salvare = True
                            file_da_salvare = True
                            gestione_intervento(Element3, Element2)
                        elif Element3.tag not in tag_da_tenere:
                            Element2.remove(Element3)
                    # se la fase contiene solo elementi che sono stati eliminati, viene eliminata anch'essa
                    if fase_da_salvare is False:
                        Element.remove(Element2)
                elif Element2.tag not in tag_da_tenere:
                    Element.remove(Element2)
            # se il dibattito contiene solo elementi che sono stati eliminati, viene eliminato anch'esso
            if da_salvare is False:
                seduta[1].remove(Element)
        else: gestione_intervento(Element, seduta[1])
    if file_da_salvare is True:
        seduta_albero = etree.ElementTree(seduta)
        seduta_albero.write('Resoconti/legislatura_{}/seduta_{}.xml'.format(num_legislatura,num_seduta), encoding ='utf-8')
    else:
        print("Seduta {} non salvata perché priva di interventi.".format(num_seduta))